In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import RegexpTokenizer
stop = stopwords.words('english')
#!pip install scispacy
import spacy
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
nlp = spacy.load("en_core_sci_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
articles_df = pd.read_csv('gdrive/My Drive/COVID-19 Kaggle Challenge/coviddata_trials.csv')[['title', 'abstract', 'body_text']]
articles_df

,title,abstract,body_text
0,Are We Ready Coronavirus Disease 2019 Arrivin...,NaN,As part of mitigation strategies during the co...
1,Metformin Inhibits Proliferation Human<br>Thy...,Objective: To uncover the potential effect of ...,Thyroid cancer is the most common malignancy i...
2,CrossCountry Comparison Case Fatality Rates<b...,NaN,"A novel coronavirus has spread through China, ..."
3,Extracorporeal membrane oxygenation support<b...,NaN,The 2019 novel coronavirus disease has spread ...
4,Management Adenoviral Keratoconjunctivitis<br...,Human adenovirus (HAdV) is the most common cau...,Human adenovirus (HAdV) is the most common cau...
...,...,...,...
4133,Anesthetic Management Patients COVID 19<br>In...,Interventions: Anesthetic management and infec...,"A NOVEL, ongoing outbreak of pneumonia was rep..."
4134,Study Infectious Agents Respiratory Diseases,NaN,"A few years ago, it was possible to diagnose a..."
4135,Antiviral effect Chinese herbal prescription<...,Ethnopharmacological relevance: The Chinese He...,GH is one of the most prevalent sexually trans...
4136,Full Spectrum Cancer Patients SARSCoV2<br>Inf...,NaN,Madam -[AQ1]Liang and colleagues [1] first rep...


In [0]:
articles_df['body_text'] = articles_df['body_text'].apply(lambda x: " ".join(x.lower() if x.lower() in stop else x for x in x.split()))
articles_df['body_text'].head()

0    as part of mitigation strategies during the co...
1    Thyroid cancer is the most common malignancy i...
2    a novel coronavirus has spread through China, ...
3    the 2019 novel coronavirus disease has spread ...
4    Human adenovirus (HAdV) is the most common cau...
Name: body_text, dtype: object

In [0]:
articles_df['body_text'] = articles_df['body_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
articles_df['body_text'].head()

0    part mitigation strategies coronavirus disease...
1    Thyroid cancer common malignancy endocrine-rel...
2    novel coronavirus spread China, originating ci...
3    2019 novel coronavirus disease spread rapidly ...
4    Human adenovirus (HAdV) common cause infection...
Name: body_text, dtype: object

In [0]:
nltk_tokenizer = RegexpTokenizer('[[\d]+,]+[\d]+|[[\w]-]+[\w]+|\$*[\d]+\.[\d]+|\S+')
tokenized_series = articles_df['body_text'].apply(nltk_tokenizer.tokenize)
tokenized_series

0       [part, mitigation, strategies, coronavirus, di...
1       [Thyroid, cancer, common, malignancy, endocrin...
2       [novel, coronavirus, spread, China,, originati...
3       [2019, novel, coronavirus, disease, spread, ra...
4       [Human, adenovirus, (HAdV), common, cause, inf...
                              ...                        
4133    [NOVEL,, ongoing, outbreak, pneumonia, reporte...
4134    [years, ago,, possible, diagnose, viral, infec...
4135    [GH, one, prevalent, sexually, transmitted, di...
4136    [Madam, -[AQ1]Liang, colleagues, [1], first, r...
4137    [Diarrhea, caused, coronaviruses, key, contrib...
Name: body_text, Length: 4138, dtype: object

In [0]:
new_series = []
for value in tokenized_series.values:
  new_series.append([str(e) for ent in nlp(" ".join(value)).ents for e in ent])
new_series

Output hidden; open in https://colab.research.google.com to view.

In [0]:
token_series = pd.Series(new_series)

In [0]:
token_series

0       [mitigation, coronavirus, disease, COVID-19, o...
1       [Thyroid, cancer, malignancy, endocrine-relate...
2       [coronavirus, spread, China, city, Wuhan, deat...
3       [novel, coronavirus, disease, Hubei, province,...
4       [Human, adenovirus, HAdV, infection, ocular, s...
                              ...                        
4133    [NOVEL, outbreak, pneumonia, Wuhan, city, Hube...
4134    [years, diagnose, viral, infection, clinical, ...
4135    [GH, sexually, transmitted, diseases, people, ...
4136    [Madam, colleagues, nationwide, analysis, pati...
4137    [Diarrhea, coronaviruses, contributor, morbidi...
Length: 4138, dtype: object

In [0]:
import gensim
dictionary = gensim.corpora.Dictionary(token_series)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in token_series]

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [0]:
for topic_number in range(10):
  for term, frequency in lda_model.show_topic(topic_number, topn=10):
    print(f'{term:20} {round(frequency, 3):.3f}')
  print('----------------------------')

COVID-19             0.015
patients             0.015
cases                0.009
data                 0.006
health               0.006
China                0.005
SARS-CoV-2           0.005
infection            0.005
license              0.005
disease              0.005
----------------------------
cells                0.007
virus                0.007
viral                0.006
infection            0.006
patients             0.005
viruses              0.004
protein              0.004
les                  0.004
human                0.003
data                 0.003
----------------------------
virus                0.005
data                 0.004
license              0.004
disease              0.004
patients             0.004
viral                0.003
model                0.003
cases                0.003
cells                0.003
COVID-19             0.003
----------------------------
patients             0.011
infection            0.006
study                0.006
disease              0

In [0]:
nltk_tokenizer = RegexpTokenizer('[[\w]-]+[\w]+|\$*[\d]+\.[\d]+|[, ]+')
nltk_tokenizer.tokenize('Hi there, 35,000 mate.')

[' ', ', ', ',', ' ']